In [43]:
import pandas as pd 
from datetime import datetime


In [1]:
!pip install lightgbm 

In [4]:
train_set = pd.read_csv('Bases/train.csv')
test_set = pd.read_csv('Bases/test.csv')

In [15]:
df = train_set.append(test_set)
print(df.shape)
df.head(2)

(1309, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [17]:
#df.isna().sum()
df['Embarked'] = df['Embarked'].fillna("S")
df['Cabin'] = df['Cabin'].fillna("U")
df['Cabin'] = df['Cabin'].str[0]
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Fare'] = df['Fare'].fillna(df['Fare'].mean())

df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,U,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C


In [19]:
df.drop(['Name','Ticket'],axis=1,inplace=True)

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
to_labeling = ['Cabin','Embarked','Sex']
df[to_labeling] = df[to_labeling].apply(label_encoder.fit_transform)
df.head(2)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0.0,3,1,22.0,1,0,7.2500,8,2
1,2,1.0,1,0,38.0,1,0,71.2833,2,0


In [21]:
data_train = df[df['PassengerId'].isin(train_set['PassengerId'])].copy()
data_train['Survived'] = data_train['Survived'].astype(int)
data_train = data_train.set_index('PassengerId')
##################3
data_test = df[df['PassengerId'].isin(test_set['PassengerId'])].copy()
data_test = data_test.set_index('PassengerId')


In [23]:
data_train

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
PassengerId,,,,,,,,,
1,0,3,1,22.000000,1,0,7.2500,8,2
2,1,1,0,38.000000,1,0,71.2833,2,0
3,1,3,0,26.000000,0,0,7.9250,8,2
4,1,1,0,35.000000,1,0,53.1000,2,2
5,0,3,1,35.000000,0,0,8.0500,8,2
...,...,...,...,...,...,...,...,...,...
887,0,2,1,27.000000,0,0,13.0000,8,2
888,1,1,0,19.000000,0,0,30.0000,1,2
889,0,3,0,29.881138,1,2,23.4500,8,2


In [27]:
import lightgbm as lgb
feature_cols = data_train.columns.drop('Survived')

dtrain = lgb.Dataset(data_train.loc[:700,feature_cols], label=data_train['Survived'][:700])
dvalid = lgb.Dataset(data_train.loc[700:,feature_cols], label=data_train['Survived'][700:])

In [32]:
param = {'num_leaves': 64, 'objective': 'binary'}
param['metric'] = 'auc'
num_round = 100
bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid],  verbose_eval=False)

[LightGBM] [Info] Number of positive: 271, number of negative: 429
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000413 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 209
[LightGBM] [Info] Number of data points in the train set: 700, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.387143 -> initscore=-0.459338
[LightGBM] [Info] Start training from score -0.459338
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

In [45]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np 
model = LGBMClassifier()
cv = RepeatedStratifiedKFold(n_splits=8, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, data_train[feature_cols], data_train['Survived'], scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f +- (%.3f)' % (np.mean(n_scores), np.std(n_scores)))
model.fit(data_train[feature_cols], data_train['Survived'])
data_test['Survived'] = model.predict(data_test[feature_cols])
data_test[['Survived']].reset_index().to_csv("Submissions/" + str(datetime.today().date()) + " LightGBM.csv",index=False)

Accuracy: 0.815 +- (0.041)
